In [1]:
import ML
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
ML.showAllVariables()
%matplotlib inline

# from fastai.imports import *
# from fastai.structured import *
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os

D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Any results you write to the current directory are saved as output.

bakery_data=pd.read_csv("BreadBasket_DMS.csv")

bakery_data['Date Time']=bakery_data['Date']+" "+bakery_data['Time']

bakery_data=bakery_data.drop(['Date','Time'],axis=1)

# bakery_data['TID'] = bakery_data.index
# bakery_data.set_index(bakery_data['TID'],inplace=True)

bakery_data = bakery_data.drop(['Date Time'],axis=1)
bakery_data.Item.count()

21293

In [3]:
bakery_data = bakery_data[~bakery_data['Item'].str.contains('NONE')]
bakery_data

,Transaction,Item
0,1,Bread
1,2,Scandinavian
2,2,Scandinavian
3,3,Hot chocolate
4,3,Jam
5,3,Cookies
6,4,Muffin
7,5,Coffee
8,5,Pastry
9,5,Bread


In [4]:
tdl = []
for i in range(1,bakery_data.Transaction.count()+1):
    tdf = bakery_data[bakery_data.Transaction == i]
    l = set()
    for j in range(0,tdf.Transaction.count()):
        l.add(tdf.Item.iloc[j])
    if len(l)>0:
        tdl.append(list(l))
    else:
        tdl.append(None)
        
col = ['items']
TDB = pd.DataFrame({"items":tdl},columns=col)
TDB = TDB.dropna()
TDB

,items
0,[Bread]
1,[Scandinavian]
2,"[Cookies, Hot chocolate, Jam]"
3,[Muffin]
4,"[Bread, Pastry, Coffee]"
5,"[Medialuna, Pastry, Muffin]"
6,"[Tea, Medialuna, Pastry, Coffee]"
7,"[Bread, Pastry]"
8,"[Bread, Muffin]"
9,"[Medialuna, Scandinavian]"


In [5]:
def loadDataSet():    
    return TDB['items'].tolist()

In [98]:
from sys import exit, exc_info
from itertools import combinations
from collections import defaultdict
from time import clock
import copy


def print_results(ck_list, transactions):
	'''prints the results of the algorithm according to the standard specified
	in the documentation of the task. The confidence values are first rounded
	and then truncated
	'''
	print('-----result--------\r\n')
	for idx, itemsets in enumerate(ck_list):
		if len(itemsets) == 0:
			continue
		print('{}-itemsets :'.format(idx + 1))
		formatted_itemsets = []
		for itemset, freq in itemsets.items():
			support = float(freq) / len(transactions)
			formatted_itemsets.append((','.join(sorted(map(str, itemset))),
									   round(support, 3)))
		sorted_itemsets = sorted(formatted_itemsets,
								 key=lambda tup: (-tup[1], tup[0]))
		for itemset, support in sorted_itemsets:
			print('[', itemset, '] (support : {0:.3f})'.format(support))

		print()

def remove_itemsets_without_min_support(itemsets, min_sup, transactions):
	'''remove the itemsets that don't have the minimum support and, if given,
	the minimum value for atm
	'''
	for itemset, freq in copy.deepcopy(itemsets).items():
		if float(freq) / len(transactions) < min_sup:
			# if min_atm is not None and len(itemset) >= 2:
			# 	mul = 1.0
			# 	for item in itemset:
			# 		mul *= ck_list[0][frozenset([item])]
			# 	atm = pow(freq, len(itemset)) / mul
			# 	if atm < min_atm:
			# 		del itemsets[itemset]
			# if min_atm is None:
			#
			del itemsets[itemset]


def generate_itemsets(ck_list, min_sup):
	'''given the trivial itemsets of length 1, this function generates the
	following itemsets using self joins, and then eliminating the itemsets
	without the minimum support
	:param ck_list: a list containing only one element which, at this
		stage, is the dictionary of itemsets of length 1
	'''
	try:
		next_candidate_item_sets = self_join(ck_list[0])
	except IndexError:
		return
	while (len(next_candidate_item_sets) != 0):
		ck_list.append(defaultdict(int))
		for idx, item_set in enumerate(next_candidate_item_sets):
			for transaction in transactions:
				if item_set.issubset(transaction):
					ck_list[-1][item_set] += 1

		remove_itemsets_without_min_support(ck_list[-1], min_sup,transactions)
		try:
			next_candidate_item_sets = self_join(ck_list[-1])
		except IndexError:
			break


def build_k_minus_one_members_and_their_occurrences(itemsets, k):
	'''in order to self join the itemsets, they must be sorted in lexical
	order. Then, all the unique sets of k-1 members must be idenfified and
	and the itemsets of length k in which they appear, associated
	:param itemsets: list of itemsets
	:param k: length of the itemsets
	:returns k_minus_one_members_and_occurrences: dictionary where keys are
		k-1 members, and values are itemsets of length k, where the members in
		its key appear
	'''
	k_minus_one_members_and_occurrences = defaultdict(list)
	for itemset in itemsets:
		# small cheat to make a list a hashable type
		k_minus_one_members = ''.join(sorted(itemset)[:k - 1])
		k_minus_one_members_and_occurrences[k_minus_one_members]. \
			append(itemset)
	return k_minus_one_members_and_occurrences


def generate_itemsets_from_kmomo(kmomo):
	'''here we identify pairs with the same first k-1 members and produce a new
	itemset of length k+1. Given the kmomo dictionary, this task is easy as we
	simply generate the combinations of size 2 of the associated itemsets where
	they appear
	:param kmomo: dictionary where keys are k-1 members, and values are
		itemsets of length k, where the members in its key appear
	:returns new_itemsets: list of itemsets containing k+1 members
	'''
	new_itemsets = []
	for k_minus_one_members, occurrences in copy.deepcopy(kmomo).items():
		if len(occurrences) < 2:
			# delete those k_minus_one_members that have only one occurrence
			del kmomo[k_minus_one_members]
		else:  # generate the new itemsets
			for combination in combinations(occurrences, 2):
				union = combination[0].union(combination[1])
				new_itemsets.append(union)
	return new_itemsets


def self_join(itemsets):
	'''generates itemsets efficiently by selfjoining the itemsets
	:param itemsets: list of itemsets
	:returns new_itemsets: list of itemsets containing l+1 members
	'''
	itemsets = itemsets.keys()  # we are only interested on the itemsets
	# themselves, not the frequencies
	#     type(itemsets)
	#     k = 1
	k = len(list(itemsets)[0])  # length of the itemsets
	# making sure all the itemsets have the same length
	assert (all(len(itemset) == k for itemset in itemsets))
	kmomo = build_k_minus_one_members_and_their_occurrences(itemsets, k)
	return generate_itemsets_from_kmomo(kmomo)


def build_one_consequent_rules(itemset, freq, ck_list):
	'''builds one-consequent rules based on an itemset and its frequency, but
	most importantly, it also generates the accurate consequents that will be
	used to generate the two-consequent rules. In other words "if one or other
	of the single-consequent rules does not hold, there is no point in
	considering the double-consequent one" [Witten et. al., Data mining :
	practical machine learning tools and techniques]
	:param itemset: an itemset
	:param freq: frequency of the above mentioned itemset
	:param ck_list: dictionaries of itemsets of all lengths
	'''
	accurate_consequents = []
	rules = []
	for combination in combinations(itemset, 1):
		consequent = frozenset(combination)
		antecedent = itemset - consequent
		ant_len_itemsets = ck_list[len(antecedent) - 1]
		conf = float(freq) / ant_len_itemsets[antecedent]
		if conf >= min_conf:
			accurate_consequents.append(consequent)
			rules.append(((antecedent, consequent), conf))
	return accurate_consequents, rules


def build_n_plus_one_consequent_rules(itemset, freq, accurate_consequents,
									  ck_list):
	'''similarly to the build_one_consequent_rules method, this one generates
	the n plus one consequent rules, the same way the former method does
	:param itemset: an itemset
	:param freq: frequency of the above mentioned itemset
	:param accurate_consequents: the accurate consequents generated in the
		former method
	:param ck_list: dictionaries of itemsets of all lengths
	'''
	rules = []
	consequent_length = 2
	while (len(accurate_consequents) != 0 and
				   consequent_length < len(itemset)):
		new_accurate_consequents = []
		for combination in combinations(accurate_consequents, 2):
			consequent = frozenset.union(*combination)
			if len(consequent) != consequent_length:
				# combined itemsets must share n-1 common items
				continue
			antecedent = itemset - consequent
			ant_len_itemsets = ck_list[len(antecedent) - 1]
			conf = float(freq) / ant_len_itemsets[antecedent]
			if conf >= min_conf:
				new_accurate_consequents.append(consequent)
				rules.append(((antecedent, consequent), conf))
		accurate_consequents = new_accurate_consequents
		consequent_length += 1
	return rules


def generate_rules(itemsets, min_conf, ck_list):
	'''
	:param itemsets: itemsets of same lenght to be used for generating subsets
	:param min_conf: minimum confidence
	:param ck_list: dictionaries of itemsets of all lengths
	:returns rules: list of rules in the form of
		[((antecedent, consequent), confidence), ...]
	'''
	rules = []
	for itemset, freq in itemsets.items():
		accurate_consequents, new_rules = \
			build_one_consequent_rules(itemset, freq, ck_list)
		rules += new_rules
		# 2-item-itemsets only produce 1-consequent rules,
		# no need to go further with those
		if len(itemset) <= 2:
			continue

		rules += build_n_plus_one_consequent_rules(itemset, freq,
												   accurate_consequents,
												   ck_list)
	return list(set(rules))


def printRules(ck_list,min_conf):
	# generating rules
	rules = []
	for itemsets in list(reversed(ck_list))[:-1]:
		rules += generate_rules(itemsets, min_conf, ck_list)

	print('RULES')
	formatted_rules = [(','.join(sorted(map(str, rule[0]))) + ' -> ' +
						','.join(sorted(map(str, rule[1]))),
						round(acc, 3))
					   for rule, acc in rules]
	sorted_rules = sorted(formatted_rules, key=lambda tup: (-tup[1], tup[0]))
	for rule, acc in sorted_rules:
		print('Rule:[{0}] , confidence: {1:.3f}'.format(rule, acc))

In [99]:
import time
min_sup = 0.001
min_conf = 0.5
# min_atm = None
starttime = time.time()

transactions = loadDataSet()
# defaultdict(function_factory)構建的是一個類似dictionary的物件，其中keys的值，自行確定賦值，
# 但是values的類型，是function_factory的類實例，而且具有預設值
ck_list = [defaultdict(int)]

c1 = ck_list[0]
# generate itemsets of length 1
for transaction in transactions:
    for item in transaction:
        c1[frozenset([item])] += 1  # 計算C1中的item frequence
remove_itemsets_without_min_support(c1, min_sup, transactions)

# generate itemsets of length > 1
generate_itemsets(ck_list, min_sup)
print_results(ck_list, transactions)
endtime = time.time()
print("It takes %d milliseconds to find the above  patterns\n\n" % ((endtime - starttime) * 1000))
printRules(ck_list, min_conf)

-----result--------

1-itemsets :
[ Coffee ] (support : 0.478)
[ Bread ] (support : 0.327)
[ Tea ] (support : 0.143)
[ Cake ] (support : 0.104)
[ Pastry ] (support : 0.086)
[ Sandwich ] (support : 0.072)
[ Medialuna ] (support : 0.062)
[ Hot chocolate ] (support : 0.058)
[ Cookies ] (support : 0.054)
[ Brownie ] (support : 0.040)
[ Farm House ] (support : 0.039)
[ Juice ] (support : 0.039)
[ Muffin ] (support : 0.038)
[ Alfajores ] (support : 0.036)
[ Scone ] (support : 0.035)
[ Soup ] (support : 0.034)
[ Toast ] (support : 0.034)
[ Scandinavian ] (support : 0.029)
[ Truffles ] (support : 0.020)
[ Coke ] (support : 0.019)
[ Spanish Brunch ] (support : 0.018)
[ Baguette ] (support : 0.016)
[ Fudge ] (support : 0.015)
[ Jam ] (support : 0.015)
[ Tiffin ] (support : 0.015)
[ Mineral water ] (support : 0.014)
[ Chicken Stew ] (support : 0.013)
[ Jammie Dodgers ] (support : 0.013)
[ Hearty & Seasonal ] (support : 0.011)
[ Salad ] (support : 0.010)
[ Frittata ] (support : 0.009)
[ Smoothies 